# Notebook: Working with AWS (S3, SQS, Python SDK) and Our PostgreSQL Database:

## **Setup:**

In [2]:
# Import external modules, packages, libraries we will use:
from dotenv import load_dotenv
import json
import os

# # Import internal modules, packages, libraries for this project:
# from .aws_sqs_functions import sqs_queue_get_next_item
# from .aws_s3_functions import s3_download_file
# from .postgresql_db_functions import get_video_info

In [ ]:
# Get access info from .env file:
load_dotenv()

S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

## **Functions for TeamReel Project:**

In [ ]:
def get_next_video():
    """
    Gets the next newly uploaded video from our queue, downloads it to the
    project folder, queries our database for information about that video,
    prompt and user, and returns that info in a Python dictionary.

    (1) Checks our TeamReel AWS SQS queue for new messages (our S3 bucket
    sends a new message to SQS everytime any user uploads a new video);
    (2) Downloads the raw video file from S3 to your project directory; and
    (3) Gets relevant info about that video, prompt and user from our PostgreSQL DB.

    Returns
    -------
    - If queue has >=1 messages in it: A dictionary with all relevant info
    about that video, prompt, and user from our DB.
    - If queue is empty: String: "No messages in queue."
    """

    # Get next message from SQS queue
    # (messages are auto-added by our S3 bucket each time a new video is uploaded to our S3 bucket):
    message = sqs_queue_get_next_item()

    # Check to see if queue is empty:
    if message == "No messages in queue.":
        return "No messages in queue."

    # If queue has messages (is not empty):
    else:

        # Message info, for deleting it after processing below:
        receipt_handle = message['ReceiptHandle']
        message_id_sqs = message['MessageId']

        # Get video's S3 key and filename from the SQS message:
        video_s3_key = json.loads(message['Body'])['Records'][0]['s3']['object']['key']
        video_s3_filename = video_s3_key.split('/')[-1]

        # -----------------------------------------------------------------------

        # Downloads the raw video file from the S3 bucket to your project directory:
        s3_download_file(bucket=S3_BUCKET_NAME,
                         filename=video_s3_filename,
                         key=video_s3_key)

        video_info = get_video_info(video_s3_key)

        # Delete the message now that we have received and processed it:
        sqs_delete_message_from_queue(receipt_handle=receipt_handle)

        return video_info

## Pipeline: Testing Above Pipeline:

In [16]:
# # This will call the above functions to:
# # (1) get the next message in the SQS queue (from S3 notification of any new video upload); 
# # (2) download the raw video file to the project directory; 
# # (3) delete the message in our SQS queue after processing it; 
# # (4) return a dictionary with info about that video, prompt and user from our DB:

# video_info = get_next_video()
# video_info

{'video': {'video_id': 115,
  'title': 'HELLO WORLD',
  's3_key': 'videos/ALPACAVID-wHgVXLxaK.webm',
  's3_filename': 'ALPACAVID-wHgVXLxaK.webm',
  'created_at': datetime.datetime(2020, 3, 19, 20, 24, 24, 129000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 3, 19, 20, 24, 24, 129000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 45,
  'question': 'What is you cant think of a question to ask? is there somewhere where you can go to for examples?'},
 'user': {'user_id': 143,
  'first_name': 'Carlos',
  'last_name': 'Hernandez',
  'name': 'Carlos Hernandez',
  'username': 'carloshernandez'}}